In [22]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
data  = pd.read_csv('./age_gender.csv')
data['pixels']=data['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))

data.head(10)

In [ ]:
data.shape

In [ ]:
data.isna().sum()

In [ ]:
data['pixels'] = data['pixels'].apply(lambda x: x/255)

In [ ]:
X = np.array(data['pixels'].tolist())
X = X.reshape(X.shape[0], 48, 48, 1)

In [ ]:
plt.figure(figsize=(20,20))
for i in range(20):
    num = np.random.randint(200)
    plt.subplot(5,5,(i%25)+1)
    dig = i*num
    plt.imshow(X[dig])
    plt.xlabel(
        "Age:"+str(data['age'].iloc[dig])+
        "  Ethnicity:"+str(data['ethnicity'].iloc[dig])+
        "  Gender:"+ str(data['gender'].iloc[dig])
    )

In [ ]:
y = data['gender']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print('Xtrain: ',X_train.shape)
print('Xtest: ',X_test.shape)
print('ytrain: ',y_train.shape)
print('ytest: ',y_test.shape)


In [ ]:
model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(48,48,1)),
    keras.layers.Conv2D(16, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(rate=0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_loss')<0.2550):
            print("\nReached 0.2000 val_loss so cancelling training!")
            self.model.stop_training = True

callback = myCallback()

model.summary()

In [ ]:
from keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train, epochs=15, validation_split=0.1, batch_size=64, callbacks=[callback])

In [ ]:
loss, acc = model.evaluate(X_test,y_test,verbose=0)
print('Test loss: {}'.format(loss))
print('Test Accuracy: {}'.format(acc))

In [ ]:
y = data['age']

X_train2, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)